# Artificial Intelligence

###  1.	Using any CSP (Constraint Satisfaction Problem) framework (using variables, value domains, and constraints), discover if the above problems can be solved and if so detail who would be in hired.




Ciara is looking for employees for her new company, which develops and provides AI based logistic software for retailers. Ciara has determined that she needs:

2 Python Programmers, 2 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.
Assume that if a person has two abilities, he or she can take on two roles in the company.

So Ciara narrowed down her selections to the following people:

- __Name & Abilities__
- Peter : Python and AI
- Juan : Web and AI
- Jim : AI and Systems
- Jane : Python and Database
- Mary : Web and Systems
- Bruce : Systems and Python
- Anita : Web and AI

### Scenario 1:

Suppose Ciara knows Python, and only has funds to hire __three__ more people.


__Variables__: The variables are the people (Peter, Juan, Jim, Jane, Mary, Bruce, Anita). Each person represents a variable that needs to be assigned a role based on their abilities.

__Value Domains__: The domain of each variable (person) is their set of abilities. For example, the domain for Peter is {"Python", "AI"}.

__Constraints__:
- Each person can be assigned only to roles matching their abilities.
- The total number of people hired should be three (excluding Ciara).
- The required roles in the company (_'constraints'_) must all be filled. This includes 1 Python Programmer, 2 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.


### Scenario 2:

Suppose Ciara and Juan become partners, with the additional funds they can now employ four more people but must employ another AI Engineer, so they need 2 Python Programmers, 3 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.

Variables and Value Domains remains the same as scenarion 1.

Constraints:

- Juan becomes Ciara's partner and will be moved to pre-selected list.
- The total number of people hired must be up to four (excluding Ciara and Juan)
- The required roles in the company `constraint` must all be filled. This includes 2 Python Programmer, 3 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.


In [4]:
from itertools import combinations
from collections import Counter
import customtkinter as ctk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import ipywidgets as widgets
import plotly.graph_objs as go
from IPython.display import display, clear_output

In [ ]:
# Adjusted variables and domain of the problem where variables = people, and domain = roles;
variables_domain = {
    "Peter": {"Python", "AI"},
    "Juan": {"Web", "AI"}, 
    "Jim": {"AI", "Systems"},
    "Jane": {"Python", "Database"},
    "Mary": {"Web", "Systems"},
    "Bruce": {"Systems", "Python"},
    "Anita": {"Web", "AI"}, 
    "Ciara": {"Python"}  
}

# Define the required skills
constraint = Counter({"Python": 2, "AI": 2, "Web": 1, "Database": 1, "Systems": 1}) # For Scenario 2 add one AI. 

# Pre-selected individuals
#pre_selected = {"Ciara", "Juan"} # For Scenarion 2
pre_selected = {"Ciara"} # For Scenarion 1

# Remove pre-selected individuals from domain
for person in pre_selected:
    # Reduce the count of required skills based on pre-selected individuals
    for skill in variables_domain[person]:
        if skill in constraint:
            constraint[skill] -= 1
    del variables_domain[person]

# Function to check if a combination of people meets the required skills
def meets_requirements(people, constraint, domains):
    skills_count = {skill: 0 for skill in constraint}
    for person in people:
        for skill in domains[person]:
            if skill in skills_count:
                skills_count[skill] += 1
    # Check if all required skills are met or exceeded
    return all(skills_count[skill] >= constraint[skill] for skill in constraint)

# Generate all possible combinations of 4 or fewer people from the remaining candidates
possible_teams = []
#for r in range(1, 5): # For scenario 2
for r in range(1, 4):  # For scenario 1
    for team in combinations(variables_domain.keys(), r):
        if meets_requirements(team, constraint, variables_domain):
            possible_teams.append(pre_selected.union(team))  # Add pre-selected individuals to the team

possible_teams

### 3.	These problems be solved using several other algorithm’s we have studied in the module. Choose one of these algorithms and discuss your answer in detail including a proof of your hypothesis in code 

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

# Variables and their domains (abilities)
variables_domain_2 = {
    "Peter": {"Python", "AI"},
    "Juan": {"Web", "AI"},
    "Jim": {"AI", "Systems"},
    "Jane": {"Python", "Database"},
    "Mary": {"Web", "Systems"},
    "Bruce": {"Systems", "Python"},
    "Anita": {"Web", "AI"}
}

# Defined roles that need to be filled, considering Ciara's contribution
role_needs = {"Python": 1, "AI": 2, "Web": 1, "Database": 1, "Systems": 1}

# Initialize the problem
prob = LpProblem("Scenario_1", LpMinimize)

# Define variables: x[person, role] is 1 if the person is assigned to the role
x = LpVariable.dicts("assignment", [(person, role) for person in variables_domain_2 for role in variables_domain_2[person]], cat='Binary')

# Define a helper variable for each person indicating if they are hired
hired = LpVariable.dicts("hired", variables_domain_2.keys(), cat='Binary')

# Objective function: Minimize the number of people hired
prob += lpSum(hired[person] for person in variables_domain_2)

# Constraints

# Ensure each role has enough people
for role in role_needs:
    prob += lpSum(x[(person, role)] for person in variables_domain_2 if role in variables_domain_2[person]) == role_needs[role]

# Link the hiring variable with the assignment variables
for person in variables_domain_2:
    prob += hired[person] * 2 == lpSum(x[(person, role)] for role in variables_domain_2[person])

# Ensure exactly 3 people are hired
prob += lpSum(hired[person] for person in variables_domain_2) == 3

# List to store all unique solutions
all_solutions = []

# Function to add constraints that block the previous solutions
def block_previous_solutions(prob, solutions, x):
    for solution in solutions:
        # Creating a unique constraint for each previous solution found
        prob += lpSum(x[person, role] for (person, role) in solution.keys() if solution[(person, role)] == 1) <= sum(solution.values()) - 1


while True:
    # Solve the problem
    prob.solve()

    # Check if a new solution is found
    if LpStatus[prob.status] == 'Optimal':
        # Extract the current solution
        current_solution = {(person, role): x[(person, role)].varValue for (person, role) in x.keys() if x[(person, role)].varValue == 1}

        # Check for uniqueness of the solution
        if current_solution not in all_solutions:
            all_solutions.append(current_solution)
            print("Solution #", len(all_solutions))
            
            # Print each person's name with their assigned roles
            assignments = {person: [] for person in variables_domain_2}
            for (person, role) in current_solution.keys():
                if current_solution[(person, role)] == 1:  # If the person is assigned to the role
                    assignments[person].append(role)  # Append the role to the person's list of roles

            # Print assignments
            for person, roles in assignments.items():
                if roles:  # If the person has roles assigned
                    print(f"{person} : {', '.join(roles)}")

            # Add a constraint to block the current solution
            block_previous_solutions(prob, [current_solution], x)

        else:
            # If the solution is already found, break the loop
            break
    else:
        # No more solutions are available
        break

print(f"Total {len(all_solutions)} unique solutions found.")


# Data Visualization


### Create interactive visualisation(s) to allow a user to explore alternate constraint scenarios




In [5]:
# Assuming these global definitions are provided earlier
skill_constraints = Counter({"Python": 2, "AI": 2, "Web": 1, "Database": 1, "Systems": 1})
employee_skills = {
    "Peter": {"Python", "AI"},
    "Juan": {"Web", "AI"},
    "Jim": {"AI", "Systems"},
    "Jane": {"Python", "Database"},
    "Mary": {"Web", "Systems"},
    "Bruce": {"Systems", "Python"},
    "Anita": {"Web", "AI"},
    "Ciara": {"Python"}
}



In [6]:
def meets_requirements(people, skill_constraints, domains):
    skills_count = Counter(skill_constraints)
    for person in people:
        for skill in domains[person]:
            if skill in skills_count:
                skills_count[skill] -= 1
    return all(count <= 0 for count in skills_count.values())

In [7]:
class TeamExplorerApp:
    def __init__(self):
        self.pre_selected = set()  # Set to keep track of pre-selected individuals
        self.init_ui_elements()

    def init_ui_elements(self):
        self.role_sliders = {}
        for role, count in skill_constraints.items():
            slider = widgets.IntSlider(value=count, min=0, max=4, description=f'{role}:')
            slider.observe(self.update_plot, names='value')
            display(slider)
            self.role_sliders[role] = slider

        self.total_people_slider = widgets.IntSlider(value=3, min=1, max=8, description='Total people to hire:')
        self.total_people_slider.observe(self.update_plot, names='value')
        display(self.total_people_slider)
        
        print("Pre-select mandatory team members:")
        for individual in employee_skills.keys():
            chkbox = widgets.Checkbox(value=False, description=individual)
            chkbox.observe(self.handle_preselect, names='value')
            display(chkbox)
        
        self.output_area = widgets.Output()
        display(self.output_area)
        
    def handle_preselect(self, change):
        individual = change.owner.description
        if change.new:  # If the checkbox is checked
            self.pre_selected.add(individual)
        else:  # If the checkbox is unchecked
            self.pre_selected.discard(individual)
        self.update_plot(None)  # Update the plot with the new pre-selected set

    def update_plot(self, change):
        updated_constraints = Counter({role: self.role_sliders[role].value for role in skill_constraints})
        total_people = self.total_people_slider.value
        possible_teams = []

        for r in range(1, total_people + 1 - len(self.pre_selected)):
            for team in combinations(set(employee_skills.keys()) - self.pre_selected, r):
                full_team = self.pre_selected.union(team)
                if meets_requirements(full_team, updated_constraints, employee_skills):
                    possible_teams.append(full_team)

        self.render_output(possible_teams)
        
    def render_output(self, possible_teams):
        with self.output_area:
            clear_output(wait=True)
            if possible_teams:
                for team in sorted(possible_teams, key=lambda x: len(x)):
                    print(f"{', '.join(sorted(team))}")
            else:
                print("No possible teams meet the requirements.")

            self.render_visualization(possible_teams)

    def render_visualization(self, possible_teams):
        # Calculate necessary data
        individual_counts = Counter([member for team in possible_teams for member in team])
        team_sizes = [len(team) for team in possible_teams]

        # Create Participant Bar Chart
        participant_fig = go.Figure()
        participant_fig.add_trace(go.Bar(x=list(individual_counts.keys()), y=list(individual_counts.values()), name="Participation"))
        participant_fig.update_layout(title_text="Participation of Employees in Teams", height=500, width=600)

        # Create Team Size Histogram with controlled x-axis and bar width
        team_size_fig = go.Figure()

        # Distinguish between single bar or multiple bars
        unique_team_sizes = list(set(team_sizes))
        if len(unique_team_sizes) == 1:  # Only one team size
            # Directly plot a bar for the single team size
            single_team_size = unique_team_sizes[0]
            team_size_fig.add_trace(go.Bar(x=[single_team_size], y=[team_sizes.count(single_team_size)], width=0.1))
            team_size_fig.update_xaxes(type='category')  # Treat x-axis as category
        else:
            # Histogram for multiple team sizes with controlled bar width
            max_team_size = max(team_sizes, default=0)
            min_team_size = min(team_sizes, default=0)

            # Adjust bin size and offset for thinner bars
            bin_size = 0.5
            offset = (1 - bin_size) / 2

            # Use histogram for team sizes
            team_size_fig.add_trace(go.Histogram(
                x=team_sizes,
                xbins=dict(
                    start=min_team_size - offset,
                    end=max_team_size + offset,
                    size=bin_size
                ),
                autobinx=False,
                marker=dict(line=dict(width=1))
            ))

            team_size_fig.update_layout(
                xaxis=dict(tickmode='array', tickvals=list(range(min_team_size, max_team_size + 1)))
            )

        team_size_fig.update_layout(title_text="Distribution of Team Sizes", height=300, width=600)

        # Show figures
        print("Team Participation:")
        participant_fig.show()
        print("Team Size Distribution:")
        team_size_fig.show()

# Running the application
app = TeamExplorerApp()


IntSlider(value=2, description='Python:', max=4)

IntSlider(value=2, description='AI:', max=4)

IntSlider(value=1, description='Web:', max=4)

IntSlider(value=1, description='Database:', max=4)

IntSlider(value=1, description='Systems:', max=4)

...

Checkbox(value=False, description='Mary')

Checkbox(value=False, description='Bruce')

Checkbox(value=False, description='Anita')

Checkbox(value=False, description='Ciara')

Output()

### Create GUI(s) to allow a user to explore alternate constraint scenarios

In [ ]:
class TeamExplorerApp:

    def __init__(self):
        ctk.set_appearance_mode("dark")  # Set dark mode for the application
        self.root = ctk.CTk()
        self.root.title("Team Constraints Explorer")
        self.root.geometry("1200x700")

        self.init_ui_elements()
        self.root.mainloop()

    def init_ui_elements(self):
        # Sliders for skill constraints with dynamic update
        self.role_sliders = {}
        for i, (role, count) in enumerate(skill_constraints.items(), start=1):
            label = ctk.CTkLabel(self.root, text=f"{role}:")
            label.place(relx=0.05, rely=0.05*i)
            slider = ctk.CTkSlider(self.root, from_=0, to=4, number_of_steps=4)
            slider.set(count)
            slider.place(relx=0.15, rely=0.05*i, relwidth=0.2)
            slider.configure(command=lambda value, role=role: self.update_plot())  # Corrected to ignore the slider value
            self.role_sliders[role] = slider

        # Slider for total people to hire with dynamic update
        self.total_people_label = ctk.CTkLabel(self.root, text="Total people to hire:")
        self.total_people_label.place(relx=0.05, rely=0.05*(len(skill_constraints)+1))
        self.total_people_slider = ctk.CTkSlider(self.root, from_=1, to=8, number_of_steps=7)
        self.total_people_slider.place(relx=0.15, rely=0.05*(len(skill_constraints)+1), relwidth=0.2)
        self.total_people_slider.configure(command=lambda value: self.update_plot())  # Corrected to ignore the slider value

        # Checkboxes for pre-selecting individuals with dynamic update
        self.pre_selected_checks = {}
        for i, individual in enumerate(employee_skills.keys(), start=0):
            var = tk.BooleanVar()
            check = ctk.CTkCheckBox(self.root, text=individual, variable=var)
            check.place(relx=0.05, rely=0.05*(len(skill_constraints)+2+i))
            var.trace('w', lambda *args, individual=individual: self.update_plot())  # Ensure no extra arguments are passed
            self.pre_selected_checks[individual] = var

        # Output Area
        self.output_area = ctk.CTkTextbox(master=self.root, height=200, width=700)
        self.output_area.place(relx=0.33, rely=0.55)
        
    def update_plot(self):
            updated_constraints = Counter({role: int(self.role_sliders[role].get()) for role in skill_constraints})
            total_people = int(self.total_people_slider.get())
            possible_teams = []

            pre_selected = {name for name, var in self.pre_selected_checks.items() if var.get()}

            for r in range(1, total_people + 1 - len(pre_selected)):
                for team in combinations(set(employee_skills.keys()) - pre_selected, r):
                    full_team = pre_selected.union(team)
                    if meets_requirements(full_team, updated_constraints, employee_skills):
                        possible_teams.append(full_team)

            self.output_area.delete(1.0, tk.END)
            if possible_teams:
                for team in sorted(possible_teams, key=lambda x: len(x)):
                    self.output_area.insert(tk.END, f"{', '.join(sorted(team))}\n")
            else:
                self.output_area.insert(tk.END, "No possible teams meet the requirements.\n")

            self.update_visualization(possible_teams)
            
    def update_visualization(self, possible_teams):
        if not hasattr(self, "plot_window") or not self.plot_window.winfo_exists():
            self.plot_window = tk.Toplevel(self.root)
            self.plot_window.title("Visualizations")
            self.plot_window.configure(bg='#333')

        for widget in self.plot_window.winfo_children():
            widget.destroy()

        individual_counts = Counter([member for team in possible_teams for member in team])
        team_sizes = [len(team) for team in possible_teams]
        total_teams = len(possible_teams)

        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        consistent_color = 'royalblue'

        # Bar chart for individual participation
        if total_teams > 0:
            axs[0].bar(individual_counts.keys(), individual_counts.values(), color=consistent_color)
            for ind, val in individual_counts.items():
                axs[0].text(ind, val, f'{(val/total_teams)*100:.1f}%', ha='center', va='bottom')

            axs[0].set_title('Participation of Employees in Teams')
            axs[0].set_xlabel('Employees')
            axs[0].set_ylabel('Number of Teams Participated In')
            axs[0].yaxis.set_major_locator(MaxNLocator(integer=True))

            # Histogram for team size distribution with thinner and separated bars
            unique_team_sizes = list(set(team_sizes))
            if len(unique_team_sizes) == 1:
                single_team_size = unique_team_sizes[0]
                axs[1].bar([single_team_size], [team_sizes.count(single_team_size)], color=consistent_color, width=0.1, align='center')
                axs[1].set_xticks([single_team_size])
            else:
                bin_width = 0.9  # smaller value means thinner bars
                bin_edges = [x - bin_width/2 for x in range(min(team_sizes), max(team_sizes)+2)]  # calculates bin edges
                axs[1].hist(team_sizes, bins=bin_edges, color=consistent_color, align='mid', rwidth=bin_width)
                axs[1].set_xticks(range(min(team_sizes), max(team_sizes)+1))

            axs[1].set_title('Distribution of Team Sizes')
            axs[1].set_xlabel('Team Size')
            axs[1].set_ylabel('Number of Teams')
        else:
            for ax in axs:
                ax.text(0.5, 0.5, 'No possible teams', ha='center', va='center', transform=ax.transAxes)
                ax.set_xticks([])
                ax.set_yticks([])

        plt.tight_layout()

        # Embedding the plot into the Tkinter window using Canvas
        canvas = FigureCanvasTkAgg(fig, master=self.plot_window)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=1)

In [ ]:
# Running the application
if __name__ == "__main__":
    app = TeamExplorerApp()